In [26]:
import pandas as pd
import numpy as np
import dateutil
from datetime import datetime, timezone, timedelta

# Reading Data

In [27]:
# Read Google Drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# Read Transaction Data
df_saida = pd.read_csv('/content/drive/My Drive/TCC_FIA_IA/saida_items2.csv', sep=';', parse_dates = ['DataSaida'], decimal =',', thousands = '.' )
# Read Inventory Data
df_estoque = pd.read_csv('/content/drive/MyDrive/TCC_FIA_IA/estoque/base_estoque.csv', sep = ',', parse_dates = ['data'])
df_estoque.drop('Unnamed: 0', inplace = True, axis = 1)

In [29]:
df_saida.DataSaida

0       2017-11-01
1       2017-11-01
2       2017-11-01
3       2017-11-01
4       2017-11-01
           ...    
18640   2022-08-31
18641   2022-08-31
18642   2022-08-31
18643   2022-08-31
18644   2022-08-31
Name: DataSaida, Length: 18645, dtype: datetime64[ns]

In [30]:
# Function to name the month

def nomear_mes(x):
    if x == 1:
        return 'JAN'
    elif x == 2:
        return 'FEV'
    elif x == 3:
        return 'MAR'
    elif x == 4:
        return 'ABR'
    elif x == 5:
        return 'MAI'
    elif x == 6:
        return 'JUN'
    elif x == 7:
        return 'JUL'
    elif x == 8:
        return 'AGO'
    elif x == 9:
        return 'SET'
    elif x == 10:
        return 'OUT'
    elif x == 11:
        return 'NOV'
    else:
        return 'DEZ'

In [31]:
# Add year, month and date in dataframe
df_saida = (

    df_saida
    .assign(ano = df_saida.DataSaida.dt.year)
    .assign(mes = df_saida.DataSaida.dt.month)
    .assign(dia = df_saida.DataSaida.dt.day)
    .assign(nome_mes = lambda x:x['mes'].apply(nomear_mes))

)

In [32]:
df_estoque = (
    df_estoque
    .assign(ano = df_estoque.data.dt.year)
    .assign(mes = df_estoque.data.dt.month)
    .assign(dia = df_estoque.data.dt.day)
    .assign(nome_mes = lambda x:x['mes'].apply(nomear_mes))
)

# Prepare ABT

## Prepare abt total

In [33]:
# Prepare principal ABT, in this dataset we cross the inventory dataset with the transactional dataset

df_transaction_hist = (
        df_saida
    .query('ano >= 2021')
    .groupby(['Item', 'ano', 'nome_mes', 'mes'])
    .agg(
        venfis = ('QtdSaida','sum'),
        venfin = ('ValorSaida', 'sum'),
        itens = ('Item', 'count'),
        itens_dist = ('Item', 'nunique'),
        datas = ('Qtd_Dias_1', 'sum'),
        data_ult_venda = ('DataSaida', 'max')
        )
    .reset_index()
    #.assign(recencia = lambda x:(x['data_ref']- x['data_ult_venda']).dt.days)
    .assign(chave = lambda x:x['ano'].astype(str)+ x['nome_mes'] + x['Item'])
    )

df_inventory_hist = (
        df_estoque
    .query('ano >= 2021')
    .groupby(['produto', 'ano', 'nome_mes'])
    .agg(
        estoque = ('disponivel','sum')
    )
    .reset_index()
    .assign(estoque = lambda x:x['estoque'].round(0))
    .assign(chave = lambda x:x['ano'].astype(str) + x['nome_mes'] + x['produto'])

    )

df_hist_abt = (
    df_transaction_hist
    .merge(df_inventory_hist, how = 'left', on = 'chave')
    .filter(['Item', 'ano_x', 'nome_mes_x' , 'mes' ,'venfis', 'venfin', 'itens', 'itens_dist', 'datas', 'data_ult_venda', 'city', 'estoque'])
    .rename(columns = {'ano_x':'ano', 'nome_mes_x': 'nome_mes'})
    .fillna(0)
    .assign(dia = '01')
    .assign(ano = lambda x:x['ano'].astype(str))
    .assign(mes = lambda x:x['mes'].astype(str))
    .assign(data_chave = lambda x:x['dia'] + '-' + x['mes'] + '-' + x['ano'])
    .assign(data_chave = lambda x:pd.to_datetime(x['data_chave'], dayfirst = True))
    )

df_hist_abt


,Item,ano,nome_mes,mes,venfis,venfin,itens,itens_dist,datas,data_ult_venda,estoque,dia,data_chave
0,-,2021,MAR,3,3,127.50,1,1,28.0,2021-03-29,0.0,01,2021-03-01
1,08IRA60-TC-GM3325,2022,JUN,6,10,44.01,1,1,28.0,2022-06-20,0.0,01,2022-06-01
2,11ER030M D=3,2021,MAI,5,3,27.52,1,1,28.0,2021-05-11,0.0,01,2021-05-01
3,11ER040M D=4,2021,MAI,5,10,27.52,1,1,28.0,2021-05-11,0.0,01,2021-05-01
4,11ER060M D=6,2021,MAI,5,10,27.52,1,1,28.0,2021-05-11,0.0,01,2021-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4348,XNGU100408ERML WU10PM,2021,AGO,8,10,40.00,1,1,28.0,2021-08-05,0.0,01,2021-08-01
4349,XNGU100408ERML WU10PM,2021,MAI,5,20,46.62,1,1,35.0,2021-05-10,20.0,01,2021-05-01
4350,XNGU100408ERML WU10PM,2021,MAR,3,10,29.43,1,1,28.0,2021-03-22,40.0,01,2021-03-01
4351,XNGU15T608ERALP WN25PM,2021,MAR,3,10,125.00,1,1,35.0,2021-03-25,0.0,01,2021-03-01


## Prepare abt by harvest

In [38]:
dats = ['2022-02-01']
df_abt_safras = pd.DataFrame()

In [39]:
# Prepare the dates for the loop

for i in dats:
    conver_dats = pd.to_datetime(i).date()
    data_inicio = conver_dats - dateutil.relativedelta.relativedelta(months = 12)
    data_fim = conver_dats + dateutil.relativedelta.relativedelta(months = 1)

    # Prepare features
    df_features = (
        df_hist_abt
        .query(f'data_chave >= "{data_inicio}" & data_chave < "{data_fim}"')
        .assign(data_ref_safra = pd.to_datetime(f'{data_fim}'))
        .assign(recencia = lambda df: (df['data_ref_safra'] - df['data_ult_venda']).dt.days)
        .filter(['Item', 'venfis', 'venfin', 'itens', 'itens_dist', 'datas', 'estoque', 'recencia'])
    )

    df_target = (
        df_hist_abt
        .query(f'data_chave == "{data_fim}"')
        .filter(['Item'])
        .drop_duplicates()
    )

    df_abt = (
        df_features
        .merge(df_target, on = 'Item', how = 'left', indicator = True)
        .assign(revendeu_prox_mes = lambda x: np.where(x['_merge'] == 'left_only', 0, 1))
        .assign(ref_prev = data_fim)
        

    )

    df_abt_safras = pd.concat([df_abt_safras, df_abt])


df_abt_safras.drop(['_merge'], axis = 1, inplace = True)
df_abt_safras.head()

,Item,venfis,venfin,itens,itens_dist,datas,estoque,recencia,revendeu_prox_mes,ref_prev
0,-,3,127.50,1,1,28.0,0.0,337,0,2022-03-01
1,11ER030M D=3,3,27.52,1,1,28.0,0.0,294,0,2022-03-01
2,11ER040M D=4,10,27.52,1,1,28.0,0.0,294,0,2022-03-01
3,11ER060M D=6,10,27.52,1,1,28.0,0.0,294,0,2022-03-01
4,11ER060M D=6,5,38.35,1,1,60.0,0.0,92,0,2022-03-01


In [40]:
df_abt_safras.shape

(2687, 10)

In [41]:
(
    df_abt_safras
 .query('revendeu_prox_mes == 0')
).index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2677, 2678, 2679, 2680, 2681, 2682, 2683, 2684, 2685, 2686],
           dtype='int64', length=2173)

# Export to CSV

In [44]:
df_abt_safras.to_csv('/content/drive/My Drive/TCC_FIA_IA/base_teste_marco.csv', sep = ',')